# Imports

In [1]:
import sys
sys.path.append("..")

from GraphBased.RP3betaRecommender import RP3betaRecommender
from FeatureWeighting.Cython.CFW_D_Similarity_Cython import CFW_D_Similarity_Cython, EvaluatorCFW_D_wrapper

from Base.Evaluation.Evaluator import EvaluatorHoldout

from Data_manager.Movielens_20m.Movielens20MReader import Movielens20MReader
from Data_manager.DataSplitter_k_fold import DataSplitter_Warm_k_fold

# Dataset selection and splitting (TODO: Switch to cold item dataset)

### Dataset is downloaded and stored locally if absent

In [2]:
# Selecting a dataset
dataReader = Movielens20MReader()

# Splitting the dataset. This split will produce a warm item split
# To replicate the original experimens use the dataset accessible here with a cold item split:
# https://mmprj.github.io/mtrm_dataset/index
dataSplitter = DataSplitter_Warm_k_fold(dataReader)
dataSplitter.load_data()

DataSplitter_k_fold for DataReader: Movielens_20m
	 Num items: 27278
	 Num users: 138493

	 Statistics for fold 0: n_interactions 3947336 ( 19.74%), n_items 27278 ( 100.00%), density: 1.04E-03
	 Statistics for fold 1: n_interactions 3999236 ( 20.00%), n_items 27278 ( 100.00%), density: 1.06E-03
	 Statistics for fold 2: n_interactions 4000004 ( 20.00%), n_items 27278 ( 100.00%), density: 1.06E-03
	 Statistics for fold 3: n_interactions 3999236 ( 20.00%), n_items 27278 ( 100.00%), density: 1.06E-03
	 Statistics for fold 4: n_interactions 4054451 ( 20.27%), n_items 27278 ( 100.00%), density: 1.07E-03


	 Statistics for ICM_genre: n_features 20, feature occurrences 54406, density: 9.97E-02


DataSplitter_k_fold: Done.


# Setup URM and ICM (TODO: Ignore warm items in cold items setting)

In [3]:
# Each URM is a scipy.sparse matrix of shape |users|x|items|
URM_train, URM_validation, URM_test = dataSplitter.get_holdout_split()

# The ICM is a scipy.sparse matrix of shape |items|x|features|
ICM = dataSplitter.get_ICM_from_name("ICM_genre")

# This contains the items to be ignored during the evaluation step
# In a cold items setting this should contain the indices of the warm items
ignore_items = []

DataSplitter: Generating holdout split... 
DataSplitter: Generating holdout split... done!


# Setup evaluators

In [4]:
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[5], ignore_items=ignore_items)
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[5], ignore_items=ignore_items)

# This is used by the ML model of CFeCBF to perform early stopping and may be omitted.
# ICM_target allows to set a different ICM for this validation step, providing flexibility in including
# features present in either validation or test but not in train
evaluator_validation_earlystopping = EvaluatorCFW_D_wrapper(evaluator_validation, ICM_target=ICM, model_to_use="last")

Ignoring 0 Items
Ignoring 0 Items


# CF Setup parameters (TODO: Parameter tuning)

In [5]:
# We compute the similarity matrix resulting from a RP3beta recommender
# Note that we have not included the code for parameter tuning, which should be done

cf_parameters = {'topK': 500,
                 'alpha': 0.9,
                 'beta': 0.7,
                 'normalize_similarity': True}

# CF Recommender training and evaluation

In [6]:
recommender_collaborative = RP3betaRecommender(URM_train)
recommender_collaborative.fit(**cf_parameters)

result_dict, result_string = evaluator_test.evaluateRecommender(recommender_collaborative)
print("CF recommendation quality is: {}".format(result_string))

import json
json = json.dumps(list(result_dict.values())[0], indent=4)
f = open("Results/RP3beta - Movielens20M/results.json","w")
f.write(json)
f.close()

EvaluatorHoldout: Processed 37401 ( 27.01% ) in 30.00 sec. Users per second: 1247
EvaluatorHoldout: Processed 76001 ( 54.88% ) in 1.01 min. Users per second: 1259
EvaluatorHoldout: Processed 115001 ( 83.04% ) in 1.51 min. Users per second: 1271
EvaluatorHoldout: Processed 138493 ( 100.00% ) in 1.81 min. Users per second: 1278
CF recommendation quality is: CUTOFF: 5 - ROC_AUC: 0.3642922, PRECISION: 0.2370199, PRECISION_RECALL_MIN_DEN: 0.2394507, RECALL: 0.0790500, MAP: 0.1562164, MRR: 0.4165224, NDCG: 0.1189325, F1: 0.1185587, HIT_RATE: 1.1850996, ARHR: 0.5706783, RMSE: 3.4897343, NOVELTY: 0.0017878, DIVERSITY_MEAN_INTER_LIST: 0.9400839, DIVERSITY_HERFINDAHL: 0.9880154, COVERAGE_ITEM: 0.1655180, COVERAGE_USER: 1.0000000, DIVERSITY_GINI: 0.0454186, SHANNON_ENTROPY: 7.4583214, 



# FW Setup parameters (TODO: Parameter tuning)

In [7]:
# We get the similarity matrix
# The similarity is a scipy.sparse matrix of shape |items|x|items|
similarity_collaborative = recommender_collaborative.W_sparse.copy()


# We instance and fit the feature weighting algorithm, it takes as input:
# - The train URM
# - The ICM
# - The collaborative similarity matrix
# Note that we have not included the code for parameter tuning, which should be done as those are just default parameters

fw_parameters =  {'epochs': 200,
                  'learning_rate': 0.0001,
                  'sgd_mode': 'adam',
                  'add_zeros_quota': 1.0,
                  'l1_reg': 0.01,
                  'l2_reg': 0.001,
                  'topK': 100,
                  'use_dropout': True,
                  'dropout_perc': 0.7,
                  'initialization_mode_D': 'random',
                  'positive_only_D': False,
                  'normalize_similarity': True}

# FW Recommender training and evaluation

In [8]:
import sys
sys.path.insert(0, '/Users/antoniochiappetta/OneDrive - Politecnico di Milano/PoliMi/Course - Digital Business Innovation/Project/Repo/recsys-videostreaming/src/FeatureWeighting/Cython')

recommender_fw = CFW_D_Similarity_Cython(URM_train, ICM, similarity_collaborative)
recommender_fw.fit(**fw_parameters,
                   evaluator_object=evaluator_validation_earlystopping,
                   stop_on_validation=True,
                   validation_every_n = 5,
                   lower_validations_allowed=10,
                   validation_metric="MAP")


result_dict, result_string = evaluator_test.evaluateRecommender(recommender_fw)
print("CFeCBF recommendation quality is: {}".format(result_string))

ModuleNotFoundError: No module named 'FeatureWeighting.Cython.CFW_D_Similarity_Cython_SGD'